# Multi-point stress approximation - walkthrough of tutorial
This notebook focuses on dissembling the `mpsa` notebook. The equations and code is looked at in detail

# Equation and physics

The linear elasticity problem may be represented with the following equation,
$$\nabla \cdot \sigma = - \vec{f}, \quad \vec{x} \in \Omega$$
where $\vec{f}$ is a body force, and $\sigma$ is the stress tensor. Stress is in PorePy represented as a linear function of the displacement
$$\sigma = C : \vec{u}$$

*Porepy* uses the following conventions:
* *tension* is positive. I.e.: The Cartesian component of the traction $\vec{T} = \sigma \cdot \vec{n}$, for a direction $\vec{r}$ is a positive number if the inner product $\vec{T}\cdot\vec{r}$ is positive. (This is opposite to the convection in *Fundamentals of Rock Mechanics*)
* The displacement vector given by $\vec{u}$ points from the initial state $\vec{x}$ to the final state $\vec{x}^*$, i.e. $\vec{u} = \vec{x}^* - \vec{x}$.

The system is bounded by three possible boundary conditions; Neumann, Dirichlet, or Robin conditions. Dividing the boundary into disjoint sets $\Gamma_N$, $\Gamma_D$, and $\Gamma_R$ for the types of conditions, we have
\begin{equation}
\vec{u} = g_D, \quad \vec{x} \in \Gamma_D \\
\sigma \cdot n = g_N, \quad \vec{x} \in \Gamma_N \\
\sigma \cdot n + W \vec{u} = g_R, \quad \vec{x} \in \Gamma_R
\end{equation}

## Create a non-fractured grid

In [ ]:
import numpy as np
import porepy as pp

# Create grid
n = 5
g = pp.CartGrid([n,n])
g.compute_geometry()

Next, we define the stress tensor $C$. The constutitive law,
\begin{equation}
\sigma = C : u = 2 \mu \epsilon + \lambda \text{trace} (\epsilon) I, \quad \epsilon = \frac{1}{2} (\nabla u + (\nabla u)^T)
\end{equation}
is implemented.
Here, $\sigma$ is the stress, $C$ is the linear mapping of stress and displacement (or strain). $\mu$ is the shear modulus, and $\lambda$ is the Lame parameter. $\epsilon$ is the strain.
Thus, defining $\lambda$ and $\mu$, we are able to provide the relationship between stress and strain.

According to Keilegavlen et. al. (2017), the representation of stress, outlined above, is valid only for the case of isotropic media. Otherwise, $C$ (the stiffness tensor) has to be expressed more generally.

The static momentum balance equation for an elastic medium, in Lagrangian coordinates are
\begin{equation}
\int_{\partial\Omega}\boldsymbol{T}(\boldsymbol{n})dA + \int_{\Omega} \boldsymbol{f} dV = 0
\end{equation}
where $\boldsymbol{T}(\boldsymbol{n})$ are the surface traction vectors on the boundary $\partial \Omega$ of the domain $\Omega$. Here, $\boldsymbol{n}$ is the outward facing normal vector, and $\boldsymbol{f}$ represents body forces. Under the assumption of a linear stress-strain relation and small deformation, we may write
\begin{equation}
\boldsymbol{T}(\boldsymbol{n}) = \boldsymbol{\pi}\cdot\boldsymbol{n} = (C:\epsilon)\cdot\boldsymbol{n}
\end{equation}
In this context, $\boldsymbol{\pi}$ represents "the first Piola-Kirchhoff stress tensor", $C$ is the stiffness tensor and $\boldsymbol{\epsilon}$ is the symmetric part of the strain $\boldsymbol{\epsilon} = (\nabla\boldsymbol{u} + (\nabla\boldsymbol{u})^T)/2$. In Euclidean space, under the assumption of small deformations, the Piola-Kirchhoff stress tensors are exactly the Cauchy stress tensor - and are symmetric.

The differences between the two stresses may be illuminated (slightly) by the following wikipedia page:
* https://en.wikipedia.org/wiki/Stress_measures#Nominal_stress/First_Piola-Kirchhoff_stress \
_Question:_ The wikipedia page mentions that the Kirchhoff stress is a "two-point stress tensor like the deformation gradient". Keilegavlen's article refers to $\epsilon = (\nabla u + (\nabla u)^T)/2$ as a deformation gradient. (I called it "strain", just above, referencing notation in the Rock Mechanics book). What is this? Is it important? \
Link: https://en.wikipedia.org/wiki/Two-point_tensor

In [ ]:
# Create stiffness matrix
lam = np.ones(g.num_cells)
mu = np.ones(g.num_cells)
C = pp.FourthOrderTensor(mu, lam)

Boundary conditions are set to
* Bottom boundary, dirichlet
* Otherwise, Neumann

In [ ]:
# Boundary type
dirich = np.ravel(np.argwhere(g.face_centers[1] < 1e-10))
labels = ['dir']*dirich.size
bound = pp.BoundaryConditionVectorial(g, dirich, labels)

The discretization scheme used is the multi-point stress approximation. 